### Autores:  
Blanco García, Gabriel: gabriel.blanco@cunef.edu  
Ferrín Meilán, Michelle: michelle.ferrin@cunef.edu

## Colegio Universitario de Estudios Financieros
### Máster en Data Science para Finanzas
Madrid, diciembre de 2020

# Modelo base

En esta sección se construye el modelo base, que servirá como referencia para comparar el resto de modelos, así como la regresión logística. También se construyen los Pipelines, herramienta fundamental para controlar y mantener en orden los pasos a seguir para el procesamiento de datos previo a los modelos. Los modelos se compararán en el último notebook, de selección de modelos, en el cual se decidirá cuál es el ganador y se calcularán diversas métricas del mismo

In [2]:
# Manipulación
import pandas as pd
import numpy as np

# Barajar los datos 
from sklearn.utils import shuffle

# Pipelines y preprocesado
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Modelos: baseline y logit
from sklearn.dummy import DummyClassifier 
from sklearn.linear_model import LogisticRegressionCV 

# Evaluación preliminar del modelo
from sklearn import metrics 

# Guardar modelos y pipelines
import pickle 

# Semilla de aleatoriedad
import random

# Omitir warnings
import warnings
warnings.filterwarnings('ignore')

Leemos los datos limpios

In [24]:
# Lectura de los datos limpios (carpeta clean dentro de data)
datos = pd.read_csv('../data/clean/clean_data.csv', engine='python')

### División de los datos

Antes de dividir el dataset, lo barajamos, por si los datos pudiesen tener algún tipo de orden que perjudicase la aleatoriedad de los datos

In [25]:
random.seed(1234)
datos = shuffle(datos)

Dividimos el dataset en tres tramos. El tramo train se utiliza para entrenar los modelos. El tramo de validación, se reserva para la optimización de hiperparámetros del modelo ganador. El tramo test, para evaluar los resultados de los modelos entrenados. Entrenamos con el 60% de los datos, y el otro 40% se divide entre validación y test, utilizando un 20% en cada parte

In [26]:
train, validation, test = np.split(datos.sample(frac=1, random_state=1234), 
                                   [int(0.6*len(datos)),
                                   int(0.8*len(datos))])

Es muy importatne poner el random state aquí, de lo contrario, los resultados no serán reproductibles

Efectuamos la separacion entre variables predictoras y variables dependientes para cada tramo

In [27]:
# Tramo de entrenamiento 
X_train = train.drop(['loan_status'], axis=1)
y_train = train['loan_status']

# Tramo de validacion 
X_validation = validation.drop(['loan_status'], axis=1)
y_validation = validation['loan_status']

# Tramo de test
X_test = test.drop(['loan_status'], axis=1)
y_test = test['loan_status']

Guardamos los datos en carpetas separadas

In [22]:
# Train en carpeta de train
X_train.to_csv('../data/train/X_train.csv', index=False)
y_train.to_csv('../data/train/y_train.csv', index=False, header=['loan_status'])

# Validación en carpeta de validación
X_validation.to_csv('../data/validation/X_validation.csv', index=False)
y_validation.to_csv('../data/validation/y_validation.csv', index=False, header=['loan_status'])

# Test en carpeta de test 
X_test.to_csv('../data/test/X_test.csv', index=False)
y_test.to_csv('../data/test/y_test.csv', index=False, header=['loan_status'])

### Modelo base

El modelo base o modelo es aquel que asigna la media u otro valor simple como prediccion del suceso. En nuestro caso, el modelo base imputará el valor más frecuente

Construimos el modelo base

In [28]:
# Construcción
modelo_base = DummyClassifier(strategy='most_frequent', # predice el valor más drecuente en el dataset 
                              random_state=1234)

In [29]:
# Entrenamiento 
modelo_base.fit(X_train, y_train)

DummyClassifier(random_state=1234, strategy='most_frequent')

In [30]:
# Accuracy del modelo 
modelo_base.score(X_test, y_test) # un 74.74%. Es coherente, es la proporción real 

0.7494780341630697

Guardamos el modelo base para las comparaciones

In [31]:
nombre = '../models/trained_models/modelo_base.sav'
pickle.dump(modelo_base, open(nombre, 'wb'))

Como vamos a tener que guardar modelos en numerosas ocasiones, definimos una función

In [32]:
def guardar_modelo(modelo, ubicacion):
      '''
    Función para guardar objetos de tipo modelo:
    - modelo: el objeto con el modelo entrenado.
    - ubicación: la carpeta de destino.
    Los modelos se guardan en formato .sav, en la carpeta 
    trained_models, dentro de models.
    
    '''
    pickle.dump(modelo, open(ubicacion, 'wb'))

# Construcción de los Pipelines 

Los Pipeline resultan una herramienta crucial en el proceso de Machine Learning. Facilitan el proceso, por el hecho de que todos los pasos pueden agruparse de manera secuencial en otros pasos más genericos.   

Además, al poder guardarlos como objetos, es posible volver a cargarlos, y así se optimiza el código, evitando el copiado y pegado que encarece el coste de solucionar errores. Lo más interesante de los Pipelines es que se pueden unir a los modelos, de tal manera que los datos pasan por el Pipeline y se procesan antes de entrar al modelo.  

Lo mejor de todo, al guardar el modelo, el Pipeline también se guarda, haciendo que no haya problemas a la hora de predecir sobre datos nuevos, puesto que todo el proceso de codificación e imputación se ejecuta justo antes.

En el fondo, los Pipelines no son más que una serie de instrucciones, de pasos predefinidos que se ejecutan en un orden específico. Vamos a utilizar un transformador de columnas. Dicho transformador se aplicará tanto a columnas numéricas como a columnas categóricas, distinción que deberá quedar explicitada. Para cada una de estas transformaciones, habrá que tratar los valores perdidos, escalar los datos...etc. 

In [22]:
# Transformador de variables numericas: se encargará de imputar los valores perdidos y de escalar los datos (en variables 
# numéricas)

transformador_numerico = Pipeline(steps=[
    
    # Paso 1: imputación de NA, nombre 'imputador'
    ('imputador', SimpleImputer(strategy='median')), # utilizamos la mediana para imputar 
    
    # Paso 2: escalar las variables, nombre 'escalador'
    ('escalador', StandardScaler())])

El escalado de las variables numéricas se hace para mejorar el rendimiento de los modelos.  

En cuanto a la imputación de valores perdidos, existen múltiples métodos. Uno de los más interesantes nos pareció el KNNImputer, que utiliza el algoritmo de clúster k-Nearest Neighbor para agrupar las observaciones en función de sus características, y así poder imputar el valor perdido.   

Probamos este imputador, pero a causa del volumen de datos, el tiempo de cómputo se extendía demasiado, así que finalmente optamos por utilizar el imputador simple, con mediana, para evitar el efecto de los valores exrtremos

In [23]:
# Transformador de variables categóricas: se encargará de imputar los valores perdidos de las variables categóricas y de 
# codificarlas, para que los modelos funcionen

transformador_categorico = Pipeline(steps=[
    
    # Paso 1: imputación de NA, nombre 'imputador'
    ('imputador', SimpleImputer(strategy='constant', fill_value='perdido')), # aquí hay que decirle qué queremos que ponga
    # No utilizamos kNN porque la cantidad de NA's en las categóricas eternizaria el proceso
    
    
    # Paso 2: codificación de variables categóricas, nombre 'onehot', usamos one hot encoding
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

Para el transformador categórcio, la imputación se realiza simplemente substituyendo el NA por "perdido".  

Las variables categóricas tienen que ir codificadas, para lo que empleamos el famoso One Hot Encoding, que consiste en genrear columnas nuevas y utilizar 0's y 1's para determinar cuando una observación presenta dicha característica. Hay que tener cuidado con este método, porque de una columna categórica con 5 posibles valores, generará 5 nuevas columnas (la original se elimina). Esto puede ser un problema para variables con muchas categorías, ya que pordía ensanchar el dataset, algo no deseable en Machine Learning.  

En cuanto a las categorías nuevas, el método seguido es simplemente ignorar dicha categoría

Guardamos los nombres de las variables numéricas (formatos int64 y float) y las categóricas (object) para poder 
usarlos en los pipelines

In [24]:
# Las numéricas
variables_numericas = datos.select_dtypes(include=['int64', 'float64']).columns

# Las categóricas (excluida la dependiente)
variables_categoricas = datos.select_dtypes(include=['object']).drop(['loan_status'], axis=1).columns

Construimos el preprocesador, con todas las piezas definidas antes

In [25]:
preprocesador = ColumnTransformer(
    transformers=[
        # Primer transformador, lo llamamos numericas: aplica los pasos del trasnformador numerico
        # a las variables de 'variables_numericas'
        ('numericas', transformador_numerico, variables_numericas),
        
        # Segundo trasnformador, lo llamamos categoricas: aplica los pasos del transformacor categórico
        # a las variables de 'variables_categoricas'
        ('categoricas', transformador_categorico, variables_categoricas)])

Como el Pipeline lo vamos a utilizar en distintos notebooks, lo guardamos con `pickle` para facilitar su uso, y así no  tener que repetir el código

In [26]:
nombre = '../pipelines/preprocesador.sav'
pickle.dump(preprocesador, open(nombre, 'wb'))

## Regresión logística

La regresión logística utiliza la función Sigmoide para devolver probabilidades de pertenencia a un grupo. Se ajusta por máxima verosimilitud, lo que quiere decir que en realidad se ajustan múltiples funciones Sigmoide, hasta que se  encuentre aquella que asigna a los puntos la probabilidad más cercana a su  grupo real. A continuación la implementamos

In [40]:
# Montamos el clasificador
regresion_logistica = Pipeline(steps=[
    ('preprocesador', preprocesador), # primero se pre-procesan los datos
    
    ('clasificador', LogisticRegressionCV(cv=10, # validación cruzada de 10 folds
                                          random_state=1234))])

In [41]:
# Entrenamos el modelo
regresion_logistica.fit(X_train, y_train)

Pipeline(steps=[('preprocesador',
                 ColumnTransformer(transformers=[('numericas',
                                                  Pipeline(steps=[('imputador',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('escalador',
                                                                   StandardScaler())]),
                                                  Index(['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'annual_inc_joint',
       'dti_joint', 'mort_acc'],
      dtype='object')),
                                                 ('categoricas',
                                                  Pipeline(steps=[('imputador',
                                                                   SimpleImputer(fill_value='perdido',
                                                                                 strategy='constant')),
                     

In [42]:
regresion_logistica.score(X_train, y_train) # 78.74% en entrenamiento

0.7874286909423974

In [43]:
regresion_logistica.score(X_test, y_test) # 78.73%, en test, muy similar, descartamos overfitting

0.787362476876643

Guardamos el modelo de regresión logística

In [44]:
guardar_modelo(regresion_logistica, '../models/trained_models/logistic_regression.sav')